In [26]:
%matplotlib inline
import os, math
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import numpy as np
import xml.etree.ElementTree as ET
from ipywidgets import interact, interactive

In [27]:
def f(p):
    show_nucleus = 0
    use_defaults = True
    scale_radius = 1.0
    axes_min = 0
    axes_max = 2000
    
    fname = "snapshot%08d.svg" % p
    if (os.path.isfile(fname) == False):
        print("File does not exist: ",fname)
        return

    xlist = deque()
    ylist = deque()
    rlist = deque()
    rgb_list = deque()
    tree = ET.parse(fname)
    root = tree.getroot()
    numChildren = 0
    for child in root:
        if use_defaults and ('width' in child.attrib.keys()):
          axes_max = float(child.attrib['width'])
        if child.text and "Current time" in child.text:
          svals = child.text.split()
          #title_str = "(" + str(current_idx) + ") Current time: " + svals[2] + "d, " + svals[4] + "h, " + svals[7] + "m"
        if ('id' in child.attrib.keys()):
#          print('-------- found tissue!!')
          tissue_parent = child
          break
    cells_parent = None

    for child in tissue_parent:
        if (child.attrib['id'] == 'cells'):
#      print('-------- found cells, setting cells_parent')
          cells_parent = child
          break
        
        numChildren += 1
        
    num_cells = 0
#  print('------ search cells')
    for child in cells_parent:
        for circle in child:  # two circles in each child: outer + nucleus
    #  circle.attrib={'cx': '1085.59','cy': '1225.24','fill': 'rgb(159,159,96)','r': '6.67717','stroke': 'rgb(159,159,96)','stroke-width': '0.5'}
#      print('  --- cx,cy=',circle.attrib['cx'],circle.attrib['cy'])
            xval = float(circle.attrib['cx'])
            s = circle.attrib['fill']
        
            if (s[0:3] == "rgb"):  # if an rgb string, e.g. "rgb(175,175,80)"
                rgb = list(map(int, s[4:-1].split(",")))
                rgb[:]=[x/255. for x in rgb]
            else:     # otherwise, must be a color name
                rgb_tuple = mplc.to_rgb(mplc.cnames[s])  # a tuple
                rgb = [x for x in rgb_tuple]
                
            too_large_val = 10000.
            if (math.fabs(xval) > too_large_val):
                print("bogus xval=",xval)
                break
            yval = float(circle.attrib['cy'])
            if (math.fabs(yval) > too_large_val):
                print("bogus xval=",xval)
                break

            rval = float(circle.attrib['r'])
            xlist.append(xval)
            ylist.append(yval)
            rlist.append(rval)
            rgb_list.append(rgb)
            if (show_nucleus == 0):
                break
        num_cells += 1
    print(fname,':  num_cells= ',num_cells)

    xvals = np.array(xlist)
    yvals = np.array(ylist)
    rvals = np.array(rlist)
    rgbs =  np.array(rgb_list)
    plt.xlim(axes_min,axes_max)
    plt.ylim(axes_min,axes_max)
    plt.axis('equal')   # square aspect ratio

    plt.scatter(xvals,yvals, s=rvals*scale_radius, c=rgbs)
    plt.show()
    

In [29]:
interactive_plot = interactive(f, p=(0, 100), continuous_update=False)
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot

interactive(children=(IntSlider(value=50, description='p'), Output(layout=Layout(height='300px'))), _dom_classes=('widget-interact',))